In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
#import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef 
import random
from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM
from sklearn.metrics import confusion_matrix, classification_report
from statistics import stdev, mean
import scipy.stats as st

KeyboardInterrupt: 

In [ ]:
model_name = 'albert_debiased'
model_class = 'albert'
new_model = 'albert-base-v2'
base = False
device = torch.device('cuda')

In [ ]:
def init_model(model_name, model_class, new_model, base):
    if base == False:
        base_model = AutoModelForMaskedLM.from_pretrained('models\{}'.format(model_name))
    else:
        base_model = AutoModelForMaskedLM.from_pretrained(model_name)

    for param in base_model.parameters():
        param.requires_grad = False

    model = AutoModelForSequenceClassification.from_pretrained(new_model, num_labels=2)

    if model_class == 'bert':
        model.bert.embeddings = base_model.bert.embeddings
        model.bert.encoder = base_model.bert.encoder
    elif model_class == 'albert':
        model.albert.embeddings = base_model.albert.embeddings
        model.albert.encoder = base_model.albert.encoder
    elif model_class == 'roberta':
        model.roberta.embeddings = base_model.roberta.embeddings
        model.roberta.encoder = base_model.roberta.encoder
    return model

In [ ]:
def scores(matrix, report):
    total = sum(sum(matrix))
    anti_TP = matrix[1][1]
    anti_FP = matrix[0][1]
    anti_FN = matrix[1][0]

    AIS = anti_TP / (anti_TP + anti_FN) * 100
    APS = anti_FP / (anti_FP + anti_FN) * 100
    
    SP = report['stereotype']['precision']
    SR = report['stereotype']['recall']
    SF = report['stereotype']['f1-score']
    SS = report['stereotype']['support']
    
    AP = report['anti-stereotype']['precision']
    AR = report['anti-stereotype']['recall']
    AF = report['anti-stereotype']['f1-score']
    AS = report['anti-stereotype']['support']
    
    MP = report['macro avg']['precision']
    MR = report['macro avg']['recall']
    MF = report['macro avg']['f1-score']
    MS = report['macro avg']['support']
    
    return AIS, APS, (SP, SR, SF, SS), (AP, AR, AF, AS), (MP, MR, MF, MS), report['accuracy']

In [ ]:
def results(aisL, apsL, accL, data_stereo, data_anti, data_macro, model_name):
    #AIS = round(sum(aisL) / len(aisL), 2)
    #APS = round(sum(apsL) / len(apsL), 2)
    
    aisL = np.array(aisL)
    apsL = np.array(apsL)
    accL = np.array(accL)
    
    AIS_mean = round(np.mean(aisL), 3)
    AIS_std = round(np.std(aisL), 3)
    
    APS_mean = round(np.mean(apsL), 3)
    APS_std = round(np.std(apsL), 3)
    
    accuracy = round(np.mean(accL), 3)
    acc_std = round(np.std(accL), 3)
    
    report = {"stereotype":{},"anti-stereotype":{},"macro avg":{}}
    
    s_p = np.array([x[0] for x in data_stereo])
    s_r = np.array([x[1] for x in data_stereo])
    s_f = np.array([x[2] for x in data_stereo])
    s_s = np.array([x[3] for x in data_stereo])
    
    a_p = np.array([x[0] for x in data_anti])
    a_r = np.array([x[1] for x in data_anti])
    a_f = np.array([x[2] for x in data_anti])
    a_s = np.array([x[3] for x in data_anti])
    
    report['stereotype']['precision'] = round(np.mean(s_p), 3)
    report['stereotype']['recall'] = round(np.mean(s_r), 3)
    report['stereotype']['f1-score'] = round(np.mean(s_f), 3)
    report['stereotype']['support'] = int(np.mean(s_s))
    
    report['anti-stereotype']['precision'] = round(np.mean(a_p), 3)
    report['anti-stereotype']['recall'] = round(np.mean(a_r), 3)
    report['anti-stereotype']['f1-score'] = round(np.mean(a_f), 3)
    report['anti-stereotype']['support'] = int(np.mean(a_s))
    
    s_f_std = round(np.std(s_f), 3)
    a_f_std = round(np.std(a_f), 3)

    #report['macro avg']['precision'] = round(sum(x[0] for x in data_macro) / len(data_macro), 3)
    #report['macro avg']['recall'] = round(sum(x[1] for x in data_macro) / len(data_macro), 3)
    #report['macro avg']['f1-score'] = round(sum(x[2] for x in data_macro) / len(data_macro), 3)
    #report['macro avg']['support'] = round(sum(x[3] for x in data_macro) / len(data_macro), 3)
    
    results_df = pd.read_csv('data/results.csv')
    
    names = results_df['Model Name'].to_list()
    if model_name in names:
        idx = results_df.loc[results_df['Model Name'] == model_name].index[0]
        results_df.loc[idx] = [model_name, report['stereotype']['precision'], report['stereotype']['recall'], report['stereotype']['f1-score'], s_f_std,
                               report['anti-stereotype']['precision'], report['anti-stereotype']['recall'], report['anti-stereotype']['f1-score'], a_f_std,
                               accuracy, acc_std, AIS_mean, AIS_std, APS_mean, APS_std]
    else:
        results_df.loc[len(results_df)] = [model_name, report['stereotype']['precision'], report['stereotype']['recall'], report['stereotype']['f1-score'], s_f_std,
                                           report['anti-stereotype']['precision'], report['anti-stereotype']['recall'], report['anti-stereotype']['f1-score'], a_f_std,
                                           accuracy, acc_std, AIS_mean, AIS_std, APS_mean, APS_std]
    results_df.to_csv('data/results.csv', index=False)
        
    
    print("The Anti-stereotype Identification Score (AIS) is: ", AIS_mean, " and std dev: ", AIS_std) 
    print("The Anti-stereotype Preference Score (APS) is: ", APS_mean, " and std dev: ", APS_std)
    print("The Model Accuracy is: ", accuracy)
    
    rep_df = pd.DataFrame.from_dict(report, orient='index')
    print(rep_df.head(4))

In [ ]:
train_df = pd.read_csv('data/train_data.tsv', delimiter='\t')

sentences = [sen for sen in train_df.iloc[:,0]]
labels = [int(lab) for lab in train_df.iloc[:,1]]

MAX_LEN = 128

if base == False:
    tokenizer = AutoTokenizer.from_pretrained('tokenizers\{}'.format(model_name),do_lower_case=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]

attention_masks = []
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [ ]:
SEED=3
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,labels,random_state=SEED,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=SEED,test_size=0.1)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

batch_size = 32

train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [ ]:
eval_df = pd.read_csv('data/eval_data.tsv', delimiter='\t')

sentences = [sen for sen in eval_df.iloc[:,0]]
labels = [int(lab) for lab in eval_df.iloc[:,1]]

input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]

attention_masks = []
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

eval_inputs = torch.tensor(input_ids).to(device)
eval_labels = torch.tensor(labels).to(device)
eval_masks = torch.tensor(attention_masks).to(device)

eval_data = TensorDataset(eval_inputs,eval_masks,eval_labels)
eval_sampler = RandomSampler(eval_data)
eval_dataloader = DataLoader(eval_data,sampler=eval_sampler,batch_size=batch_size)

In [ ]:
aisL = []
apsL = []
data_stereo = []
data_anti = []
data_macro = []
accL = []

# Parameters:
lr = 3e-4
adam_epsilon = 1e-6
epochs = 3

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

for train_iter in range(10):
    
    model = init_model(model_name, model_class, new_model, base)
    model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

    train_loss_set = []
    learning_rate = []

    for _ in trange(1,epochs+1,desc='Epoch'):
        batch_loss = 0

        for step, batch in enumerate(train_dataloader):
            model.train()

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs[0]

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            batch_loss += loss.item()

        avg_train_loss = batch_loss / len(train_dataloader)

        for param_group in optimizer.param_groups:
            learning_rate.append(param_group['lr'])

        train_loss_set.append(avg_train_loss)

        # Validation
        model.eval()

        eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            with torch.no_grad():
              logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            logits = logits[0].to('cpu').numpy()
            label_ids = b_labels.to('cpu').numpy()

            pred_flat = np.argmax(logits, axis=1).flatten()
            labels_flat = label_ids.flatten()
            tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
            tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)

            eval_accuracy += tmp_eval_accuracy
            eval_mcc_accuracy += tmp_eval_mcc_accuracy
            nb_eval_steps += 1
    

    # Evaluate on evaluation set
    preds = []
    for batch in eval_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            logits = model(b_input_ids, attention_mask=b_input_mask)
            pred = logits[0].to('cpu').numpy()
            pred = np.argmax(pred, axis=1)
            for p in pred:
                preds.append(p)

    preds = np.array(preds)
    eval_labels = eval_labels.cpu()
    
    pred_labels = preds.round().reshape(-1)
    matrix = confusion_matrix(eval_labels, pred_labels)
    report = classification_report(eval_labels, pred_labels, target_names=['stereotype', 'anti-stereotype'], output_dict=True)
    
    ais, aps, stereo, anti, macro, acc = scores(matrix, report)
    
    aisL.append(ais)
    apsL.append(aps)
    accL.append(acc)
    data_stereo.append(stereo)
    data_anti.append(anti)
    data_macro.append(macro)

In [ ]:
results(aisL, apsL, accL, data_stereo, data_anti, data_macro, model_name)

In [ ]:
torch.cuda.empty_cache()